# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
img_size=128

### Get validation data

In [9]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

In [10]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


## Read data

In [11]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [14]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210415_140527_gclip1.0', '20210412_70407_ddp_code_…

In [15]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210416_165837_train


## Plot Losses

In [16]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [17]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
7011,7011.0,17.0,0.147428,0.246670,0.394098,4.608768,86.681526,82.041702,0.034040,83.282349,0.210472,NaN,0.031062,3.776535,-3.331819,-4.592322,0.708583
7012,7012.0,17.0,0.178185,0.203548,0.381733,2.968736,86.621628,83.646599,-0.019990,84.159805,0.243277,NaN,0.006295,2.733747,-3.603261,-2.902947,0.674641
7013,7013.0,17.0,0.157183,0.290778,0.447961,4.934583,87.888458,82.914932,0.037083,84.710693,0.154294,NaN,0.038944,3.565050,-1.963299,-4.925843,0.687948
7014,7014.0,17.0,0.241204,0.133398,0.374602,2.716363,83.614052,80.866516,-0.056494,81.170990,0.204850,NaN,0.031172,2.186298,-4.266560,-2.596789,0.698109
7015,7015.0,17.0,0.092958,0.272954,0.365912,4.899486,86.647598,81.712646,-0.101710,83.392464,0.154121,NaN,0.035472,4.063717,-2.153260,-4.887992,0.684684
7016,7016.0,17.0,0.197577,0.182563,0.380140,3.736968,84.475792,80.713959,-0.096334,81.287300,-0.008887,NaN,0.024863,2.157656,-3.798835,-3.684473,0.664074
7017,7017.0,17.0,0.135806,0.235317,0.371123,3.812696,87.869308,84.040543,-0.139307,80.397278,-0.061628,NaN,0.016064,3.276579,-2.767221,-3.782061,0.756366
7018,7018.0,17.0,0.123043,0.135975,0.259018,3.867544,86.772034,82.868530,-0.105156,84.476944,0.122756,NaN,0.035957,3.237292,-2.837887,-3.817240,0.722587
7019,7019.0,17.0,0.201157,0.277232,0.478389,4.808825,88.347107,83.521820,-0.037264,84.239594,0.214978,NaN,0.016461,2.950036,-2.273646,-4.794010,0.718709
7020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.866249,0.114330,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [ ]:
200000/(32*)

In [17]:
# import matplotlib
# import ipympl
# ipympl.__version__,matplotlib.__version__

In [23]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-2.196044776439667


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
15270,15270.0,19.0,0.176766,0.235674,0.412440,3.798450,83.216766,79.418320,-2.403741,80.949219,-2.516074,NaN,NaN,3.807446,-2.566484,-3.759541,0.110441
15217,15217.0,19.0,0.226956,0.216811,0.443766,3.632610,87.034088,83.401482,-2.260765,83.275902,-2.496010,NaN,NaN,3.457476,-2.700920,-3.586139,0.115340
14143,14143.0,18.0,0.242963,0.272021,0.514984,4.543347,91.020683,86.477333,-2.319113,82.499435,-2.462764,NaN,NaN,5.917330,-2.649555,-4.526629,0.113639
14144,14144.0,18.0,0.207555,0.093752,0.301307,5.633799,89.032661,83.398865,-2.165644,81.368774,-2.461460,NaN,NaN,4.498636,-3.741777,-5.615694,0.113801
15458,15458.0,19.0,0.199553,0.318559,0.518112,6.184422,85.116455,78.932037,-2.129415,81.948410,-2.458336,NaN,NaN,3.896608,-1.812912,-6.180612,0.111813
15265,15265.0,19.0,0.148657,0.243852,0.392509,4.564387,87.120293,82.555908,-2.118960,84.456985,-2.454945,NaN,NaN,3.613182,-2.414433,-4.549177,0.110911
15271,15271.0,19.0,0.181842,0.155532,0.337375,3.332093,83.446037,80.113945,-2.224336,84.349365,-2.442191,NaN,NaN,2.760976,-2.952711,-3.271198,0.109091
15413,15413.0,19.0,0.123303,0.532639,0.655942,7.022793,89.481186,82.458389,-2.131046,84.034355,-2.424244,NaN,NaN,5.550435,-0.973431,-7.021367,0.110313
15269,15269.0,19.0,0.227778,0.218419,0.446198,3.640672,88.118729,84.478058,-2.392753,81.061813,-2.422213,NaN,NaN,3.589384,-3.352278,-3.593090,0.109199
15459,15459.0,19.0,0.518518,0.183222,0.701740,1.269992,84.397224,83.127235,-1.936732,83.386436,-2.411009,NaN,NaN,0.745246,-4.733952,-0.729350,0.111626


In [24]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
15270,15270.0,19.0,0.176766,0.235674,0.412440,3.798450,83.216766,79.418320,-2.403741,80.949219,-2.516074,NaN,NaN,3.807446,-2.566484,-3.759541,0.110441
15217,15217.0,19.0,0.226956,0.216811,0.443766,3.632610,87.034088,83.401482,-2.260765,83.275902,-2.496010,NaN,NaN,3.457476,-2.700920,-3.586139,0.115340
14143,14143.0,18.0,0.242963,0.272021,0.514984,4.543347,91.020683,86.477333,-2.319113,82.499435,-2.462764,NaN,NaN,5.917330,-2.649555,-4.526629,0.113639
14144,14144.0,18.0,0.207555,0.093752,0.301307,5.633799,89.032661,83.398865,-2.165644,81.368774,-2.461460,NaN,NaN,4.498636,-3.741777,-5.615694,0.113801
15458,15458.0,19.0,0.199553,0.318559,0.518112,6.184422,85.116455,78.932037,-2.129415,81.948410,-2.458336,NaN,NaN,3.896608,-1.812912,-6.180612,0.111813
15265,15265.0,19.0,0.148657,0.243852,0.392509,4.564387,87.120293,82.555908,-2.118960,84.456985,-2.454945,NaN,NaN,3.613182,-2.414433,-4.549177,0.110911
15271,15271.0,19.0,0.181842,0.155532,0.337375,3.332093,83.446037,80.113945,-2.224336,84.349365,-2.442191,NaN,NaN,2.760976,-2.952711,-3.271198,0.109091
15413,15413.0,19.0,0.123303,0.532639,0.655942,7.022793,89.481186,82.458389,-2.131046,84.034355,-2.424244,NaN,NaN,5.550435,-0.973431,-7.021367,0.110313


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
10496,10496.0,13.0,0.157497,0.203417,0.360914,5.227551,86.363708,81.136154,-0.676352,75.525787,-0.673908,NaN,NaN,4.244079,-2.894887,-5.216831,0.111376
6681,6681.0,8.0,0.109917,0.244982,0.354899,4.859572,93.692574,88.833000,0.462267,76.723579,0.879972,NaN,NaN,4.959079,-2.072416,-4.844605,0.109990
5955,5955.0,7.0,0.163442,0.265429,0.428871,3.580450,88.931419,85.350967,0.545305,76.856873,0.873281,NaN,NaN,2.494502,-4.883062,-3.533962,0.111889
11234,11234.0,14.0,0.252117,0.189475,0.441593,6.464019,91.171066,84.707047,-0.356605,77.137726,-0.178151,NaN,NaN,3.329084,-6.033804,-6.460470,0.109306
6332,6332.0,8.0,0.151615,0.139861,0.291475,3.866189,90.142334,86.276146,0.576064,77.180923,0.896008,NaN,NaN,3.000601,-3.966658,-3.824923,0.111733
11246,11246.0,14.0,0.138518,0.326866,0.465384,5.121240,91.255226,86.133987,-0.372636,77.209076,-0.490048,NaN,NaN,4.485376,-2.353694,-5.111780,0.111764
5939,5939.0,7.0,0.132219,0.363125,0.495343,8.952679,96.422394,87.469719,0.496318,77.414948,0.932011,NaN,NaN,3.221142,-9.197906,-8.952467,0.111200
11681,11681.0,14.0,0.186375,0.230638,0.417013,4.776809,84.454788,79.677979,-1.033209,77.440178,-1.117452,NaN,NaN,3.786543,-2.078054,-4.763651,0.114431


## Read stored chi-squares for images

In [25]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [26]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [27]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [28]:
df_sliced=w.result
# df_sliced

In [29]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.10293212384428, 'chi_spec1': 2.6377671026896046}
[0, 19]
{'chi_1': 0.2477671891857311, 'chi_spec1': 9.147125419962958}
[0, 19]
{'chi_1': 0.10293212384428, 'chi_spec1': 2.6377671026896046}
[array([15470, 15550, 15410, 14340]), array([14520, 12260]), array([14500, 14490])]
[12260 14340 14490 14500 14520 15410 15470 15550]


In [30]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [31]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 22)
[(15, 12260), (18, 14340), (18, 14490), (18, 14500), (18, 14520), (19, 15410), (19, 15470), (19, 15550)]


In [32]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
1225,15-12260,0.067769,2.317462,29.900444,0.052945,0.013648,0.001176,0.031751,12.259857,194.191968,15,12260
1433,18-14340,0.013687,0.922040,27.085577,0.002710,0.008945,0.002032,0.001619,17.404840,77.404388,18,14340
1448,18-14490,0.051186,0.497541,26.362005,0.033812,0.006206,0.011167,0.027219,15.234585,46.483007,18,14490
1449,18-14500,0.029953,0.484897,24.867995,0.022863,0.005457,0.001633,0.014754,14.791797,40.380687,18,14500
1451,18-14520,0.060960,1.136974,29.499259,0.054024,0.005759,0.001176,0.034898,14.316355,87.140953,18,14520
1540,19-15410,0.013129,0.926614,29.003857,0.002593,0.008048,0.002488,0.000356,13.980037,72.761683,19,15410
1546,19-15470,0.011841,1.281412,27.266224,0.001834,0.007219,0.002787,0.000263,15.545033,125.018546,19,15470
1554,19-15550,0.012448,1.599003,27.783432,0.003645,0.007380,0.001422,0.000633,15.588761,143.546205,19,15550


### Plots 2: stored data

In [33]:
def f_plot_hist_spec(df,dict_bkg,plot_type):
    
    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)
            plt.legend()
            
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:]
            print(images.shape)
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
    
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
    
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [34]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('15-12260', '18-14340', '18-14490', '…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [35]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(ip_fname,mmap_mode='r')[:1000,0,:,:]
print(s_val.shape)
f_plot_grid(s_val[100:118],cols=6,fig_size=(6,3))


(1000, 128, 128)
3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
epoch,step=20,1960
fname='/images/gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step)
img=np.load(result_dir+fname,mmap_mode='r')[:1000,0,:,:]
print(img.shape)
f_plot_grid(img[100:118],cols=6,fig_size=(6,3))

FileNotFoundError: [Errno 2] No such file or directory: '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210407_174739_new_loss_noddpcode/images/gen_img_epoch-20_step-1960.npy'